In [35]:
# https://github.com/bibliotechy/DPyLA
# https://pro.dp.la/developers/field-reference#sourceResource-subject

In [33]:
from configparser import ConfigParser
import simplejson as json

import dpla
from dpla.api import DPLA

import pprint

print 'all set.'

all set.


### REMOTE API

If you do not wish for others to see your API Key, then add a file that adheres to the specificaitons laid out in the [ConfigParser](https://docs.python.org/2/library/configparser.html) documentation.

In [32]:
config = ConfigParser()
config.read(u'data/api_keys.txt') # REQUIRES UNICODE TO AVOID DEPRECATION WARNING
api_key = config.get('dpla', 'api_key')

dpla = DPLA(api_key)

# print 'Product Key %s initalized, we are a go!' % api_key

In [33]:
#DPLA URL: https://dp.la/item/0a9d7daa792c26aa0288e23b843176ee
#ORIGINAL URL: https://dlg.usg.edu/record/gcfa_gsac_gsac492

pp = pprint.PrettyPrinter(depth=2)

item_id = "0a9d7daa792c26aa0288e23b843176ee"

record = dpla.fetch_by_id([item_id])


In [ ]:
# pp.pprint(record.items[0]['sourceResource'])

In [34]:
# pp.pprint(record.items[0]['originalRecord'])

In [31]:
print 'DPLA SUBJECTS'
pp.pprint(record.items[0]['sourceResource']['subject'])

DPLA SUBJECTS
[{u'name': u'Fiberwork--Georgia'},
 {u'name': u'Fiberwork--United States'},
 {u'name': u'Art--Georgia'},
 {u'name': u'Textile design, Abstract'}]


In [32]:
print 'ORIGINAL RECORD SUBJECTS'
pp.pprint(record.items[0]['originalRecord']['subject'])

ORIGINAL RECORD SUBJECTS
[u'Fiberwork--Georgia',
 u'Fiberwork--United States',
 u'Art--Georgia',
 u'Textile design, Abstract']
